### Import all required libraries

- *`random`*: For generating random choices. It is used to select the next note in the music sequence;


- *`mingus`*: Library for handling musical notation and MIDI files;
- *`Note`*: Represents a musical note;
- *`Bar`:* Classes for creating and managing MIDI bars of music;
- *`Track`*: Classes for creating and managing MIDI tracks of music;
- *`midi_file_out`*: Module for writing MIDI files;


- *`pygame`*: Library for creating games. It is used to play the MIDI file sounds;


- *`time`*: Used to pause execution while the music is playing.

In [1]:
import random

from mingus.containers import Note, Track, Bar
from mingus.midi import midi_file_out

import pygame

import time

pygame 2.6.0 (SDL 2.28.4, Python 3.9.19)
Hello from the pygame community. https://www.pygame.org/contribute.html


### Create an example note sequence

The created list (*`notes`*) contains all the notes in the C Major scale. The list *`notes`* serves as the pool of note to generate music.

In [2]:
notes = ['C', 'D', 'E', 'F', 'G', 'A', 'B']

### Define transition matrix for the notes

The dictionary (*`transition_matrix`*) is used to define a transition matrix. This matrix is used to map each note of the C Major scale and the next possible notes that can follow it. 

In [3]:
transition_matrix = {
    'C': ['D', 'E', 'F', 'G', 'A', 'B'],
    'D': ['C', 'E', 'F', 'G', 'A', 'B'],
    'E': ['C', 'D', 'F', 'G', 'A', 'B'],
    'F': ['C', 'D', 'E', 'G', 'A', 'B'],
    'G': ['C', 'D', 'E', 'F', 'A', 'B'],
    'A': ['C', 'D', 'E', 'F', 'G', 'B'],
    'B': ['C', 'D', 'E', 'F', 'G', 'A'],
}

### Generate music function

The function (*`generate_music`*) is used to create a sequence of musical notes. The function takes as arguments the following:
* *`start_note`* -> The starting (base) note of the sequence;
* *`length`* -> The number of notes to be generated (Default length is 20);
* *`octave`* -> The octave of the notes (Default octave is 4th).

When the function is called, the following occurs:
1. An empty *`music`* list is created to store all the generated notes.
2. The starting note of the sequence is user defined. 
3. For the desired number of notes to be generated (*`length`*):
    1. A random note is chosen from the transitional matrix. The base note is given as a key, and then from the key's value the following note is chosen at random.
    2. After choosing the next note it is used to create a *`Note`* object at the specified *`octave`*.
    3. The created note object (*`next_note_obj`* is appended to the *`music`* list.
    4. The *`current_note`* is updated.
4. The *`music`* list, containing all the predicted/generated notes, is returned. 

In [4]:
def generate_music(start_note, length=20, octave=4):
    music = []
    current_note = start_note

    for _ in range(length):
        next_note = random.choice(transition_matrix[current_note])
        next_note_obj = Note(next_note, octave)
        music.append(next_note_obj)
        current_note = next_note

    return music

### Generate a music sequence

The *`generated_music`* variable is defined by calling the *`generate_music`* function. as parameters to the function are provided the base note, the number of notes to generate (if different from the default length of 20) and the octave in which the notes to be (if different from the default 4th octave). 

The generated music sequence is printed to the user.


In [5]:
generated_music = generate_music('C', length=20, octave=2)
print("Generated music sequence:", [str(note) for note in generated_music])

Generated music sequence: ["'A-2'", "'E-2'", "'B-2'", "'G-2'", "'D-2'", "'B-2'", "'G-2'", "'E-2'", "'C-2'", "'G-2'", "'B-2'", "'E-2'", "'D-2'", "'F-2'", "'E-2'", "'A-2'", "'B-2'", "'C-2'", "'A-2'", "'F-2'"]


### Creating Track() and Bar() objects

* *`track`* is a collection of bars.
* *`bar`* is a sequence of notes.

In [6]:
track = Track()
bar = Bar()

### Add notes to the bar

Iterating over the generated *`notes`* and adding them to the current *`bar`*. Each bar contains a set number of beats (in this case 4 beats) the bar is added to the *`track`*, and a new *`bar`* is started. After iterating over all the *`notes`* the last *`bar`* is added to the *`track`* even if it is not full.

In [7]:
# Add notes to the bar with a fixed duration
for note in generated_music:
    if bar.is_full():
        track.add_bar(bar)
        bar = Bar()  # Start a new bar if the current one is full
    bar.place_notes(note, 4)  # 4 beats per note

# Add the last bar even if it's not full
track.add_bar(bar)

[None, [[[0.0, 4, ['A-2']], [0.25, 4, ['E-2']], [0.5, 4, ['B-2']], [0.75, 4, ['G-2']]], [[0.0, 4, ['D-2']], [0.25, 4, ['B-2']], [0.5, 4, ['G-2']], [0.75, 4, ['E-2']]], [[0.0, 4, ['C-2']], [0.25, 4, ['G-2']], [0.5, 4, ['B-2']], [0.75, 4, ['E-2']]], [[0.0, 4, ['D-2']], [0.25, 4, ['F-2']], [0.5, 4, ['E-2']], [0.75, 4, ['A-2']]], [[0.0, 4, ['B-2']], [0.25, 4, ['C-2']], [0.5, 4, ['A-2']], [0.75, 4, ['F-2']]]]]

### Write the Track to a MIDI file

The MIDI file is created by using *`midi_file_out.write_Track()`*. As arguments are provided:
* MIDI file name (*`midi_filename`*); 
* The created *`track`*.

In [8]:
midi_filename = "simple_melody.mid"
midi_file_out.write_Track(midi_filename, track)

True

### Playing the MIDI file

To play the MIDI file is required to: 
1. Initialize the Pygame library (*`pygame.init()`*);
2. Initialize the Pygame mixer module for playing sound (*`pygame.mixer.init()`*);
3.  Load the generated MIDI file (*`pygame.mixer.music.load(midi_filename)`*);
4. Start playing the MIDI file (*`pygame.mixer.music.play()`*);
5. Wait until the music finishes playing (*`while pygame.mixer.music.get_busy()`*);
6. Quit Pygame (*`pygame.quit()`*).

In [9]:
# Initialize Pygame and play the MIDI file
pygame.init()
pygame.mixer.init()
pygame.mixer.music.load(midi_filename)
pygame.mixer.music.play()

# Wait until the music finishes playing
while pygame.mixer.music.get_busy():
    time.sleep(1)

# Quit Pygame
pygame.quit()